# In this tutorial we will cover how to use autodp for the passive and active learning versions of PATE

## 1. The first step is to represent PATE in autodp
It is quite straightforward because it's just a sequence of Gaussian mechanisms with the same parameters.

In [1]:
from autodp.mechanism_zoo import GaussianMechanism 

import numpy as np

class PATE(GaussianMechanism):
    def __init__(self,sigma, m, Binary,name='PATE'):
        # sigma is the std of the Gaussian noise added to the voting scores
        if Binary:
            # This is a binary classification task
            sensitivity = 1
        else: # for the multiclass case, the L2 sensitivity is sqrt(2)
            sensitivity = np.sqrt(2)
        GaussianMechanism.__init__(self, sigma=sigma/sensitivity/np.sqrt(m),name=name)
        
        self.params = {'sigma':sigma}


## 2. It is straightforward to do privacy loss computation / privacy accounting

In [2]:
# Computing the privacy loss after running the algorithm

# let's say after running PATE-PSQ or ASQ with Gaussian mechanism to release m labels
# the noise added to the voted histogram (in the multi-class case), 
# or the noise added to the # of teachers who voted positive is sigma
# 
m = 100
sigma = 20

# let's say it is a binary classification task


pate_mech = PATE(sigma=20,m=m,Binary=True, name='PATE')

delta = 1e-6
eps = pate_mech.get_approxDP(delta)

print(eps,delta)


pate_mech = PATE(sigma=10,m=m,Binary=True, name='PATE')
delta = 1e-6
eps = pate_mech.get_approxDP(delta)

print(eps,delta)


2.2540846502197414 1e-06
4.886554117462211 1e-06


/usr/local/lib/python3.8/site-packages/scipy/optimize/optimize.py:2555: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


## 3. When we need to fulfill a pre-defined privacy parameter, we do privacy cabliration.

In [3]:
# Privacy calibtation:  given m,  eps,  choose sigma

from autodp.calibrator_zoo import eps_delta_calibrator

calibrate = eps_delta_calibrator()

class PATE_binary_m(PATE):
    def __init__(self,sigma, name='PATE_m'):
        PATE.__init__(self, sigma=sigma,m=100,Binary=True,name=name)

# Find the \sigma parameter that gives the following privacy guarantee
eps = 2.0
delta = 1e-6

mech1 = calibrate(PATE_binary_m,eps,delta,[0,100],name='PATE_eps=2')
print(mech1.name, mech1.params,mech1.get_approxDP(delta))


# Find the \sigma parameter that gives the following privacy guarantee
eps = 0.5
delta = 1e-6

mech2 = calibrate(PATE_binary_m,eps,delta,[0,100],name='PATE_eps=0.5')

print(mech2.name, mech2.params,mech2.get_approxDP(delta))



PATE_eps=2 {'sigma': 22.30476293897311} 1.999999977711552
PATE_eps=0.5 {'sigma': 80.57618519751232} 0.49999999740790046


The above sigma parameters will then be used in running the PATE algorithm.  For active learning-version of PATE, set $m$ to be the label-budget, rather than the number of public unlabeled data points available.

